# Exporting Landsat 8 to GeoTiff format  

> **Description**  
> The code in this notebook subsets a data cube, selects a specific set of variables, and then outputs that data into a GeoTIFF file. The goal is to be able to do external analyses of this data using other data analysis tools or GIS tools. The files would be reasonable in size, since we would restrict the region and parameters in the output.

----  

# Boiler Plate, Loading Data

> ### Import the Datacube

In [1]:
import datacube
dc = datacube.Datacube(app = 'my_app', config = '/home/localuser/.datacube.conf')

/home/localuser/Datacube/datacube_env/lib/python3.6/site-packages/psycopg2-2.7.5-py3.6-linux-x86_64.egg/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


>### Browse the available Data Cubes on the storage platform    
> You might want to learn more about what data is stored and how it is stored.


In [2]:
list_of_products = dc.list_products()
netCDF_products = list_of_products[list_of_products['format'] == 'NetCDF']
netCDF_products

,name,description,creation_time,product_type,instrument,lat,format,lon,time,platform,label,crs,resolution,tile_size,spatial_dimensions
id,,,,,,,,,,,,,,,
13,ls7_ledaps_ghana,Landsat 7 USGS Collection 1 Higher Level SR sc...,None,LEDAPS,ETM,None,NetCDF,None,None,LANDSAT_7,None,EPSG:4326,"(-0.000269494585236, 0.000269494585236)","(0.943231048326, 0.943231048326)","(latitude, longitude)"
17,ls7_ledaps_kenya,Landsat 7 USGS Collection 1 Higher Level SR sc...,None,LEDAPS,ETM,None,NetCDF,None,None,LANDSAT_7,None,EPSG:4326,"(-0.000269493, 0.000269493)","(0.99981903, 0.99981903)","(latitude, longitude)"
18,ls7_ledaps_senegal,Landsat 7 USGS Collection 1 Higher Level SR sc...,None,LEDAPS,ETM,None,NetCDF,None,None,LANDSAT_7,None,EPSG:4326,"(-0.000271152, 0.00027769)","(0.813456, 0.83307)","(latitude, longitude)"
16,ls7_ledaps_sierra_leone,Landsat 7 USGS Collection 1 Higher Level SR sc...,None,LEDAPS,ETM,None,NetCDF,None,None,LANDSAT_7,None,EPSG:4326,"(-0.000269494585236, 0.000269494585236)","(0.943231048326, 0.943231048326)","(latitude, longitude)"
19,ls7_ledaps_tanzania,Landsat 7 USGS Collection 1 Higher Level SR sc...,None,LEDAPS,ETM,None,NetCDF,None,None,LANDSAT_7,None,EPSG:4326,"(-0.000271277688070265, 0.000271139577954979)","(0.999929558226998, 0.999962763497961)","(latitude, longitude)"
31,ls7_ledaps_vietnam,Landsat 7 USGS Collection 1 Higher Level SR sc...,None,LEDAPS,ETM,None,NetCDF,None,None,LANDSAT_7,None,EPSG:4326,"(-0.000269494585236, 0.000269494585236)","(0.943231048326, 0.943231048326)","(latitude, longitude)"
9,ls8_lasrc_ghana,Landsat 8 USGS Collection 1 Higher Level SR sc...,None,LaSRC,OLI_TIRS,None,NetCDF,None,None,LANDSAT_8,None,EPSG:4326,"(-0.000269494585236, 0.000269494585236)","(0.943231048326, 0.943231048326)","(latitude, longitude)"
10,ls8_lasrc_kenya,Landsat 8 USGS Collection 1 Higher Level SR sc...,None,LaSRC,OLI_TIRS,None,NetCDF,None,None,LANDSAT_8,None,EPSG:4326,"(-0.000271309115317046, 0.00026957992707863)","(0.999502780827996, 0.999602369607559)","(latitude, longitude)"
11,ls8_lasrc_senegal,Landsat 8 USGS Collection 1 Higher Level SR sc...,None,LaSRC,OLI_TIRS,None,NetCDF,None,None,LANDSAT_8,None,EPSG:4326,"(-0.000271152, 0.00027769)","(0.813456, 0.83307)","(latitude, longitude)"


>### Pick a product  
>Use the platform names from the previous block to select a small Data Cube. The data_access_api utility will give you lat, lon, and time bounds of your Data Cube.   

In [3]:
import utils.data_cube_utilities.data_access_api as dc_api  
api = dc_api.DataAccessApi(config = '/home/localuser/.datacube.conf')

platform = "LANDSAT_8"
product = "ls8_lasrc_ghana"

# Get product extents
prod_extents = api.get_query_metadata(platform=platform, product=product, measurements=[])#.get_full_dataset_extent(platform = platform, product = product)

latitude_extents = prod_extents['lat_extents']
print("Lat bounds:", latitude_extents)
longitude_extents = prod_extents['lon_extents']
print("Lon bounds:", longitude_extents)
time_extents = list(map(lambda time: time.strftime('%Y-%m-%d'), prod_extents['time_extents']))
print("Time bounds:", time_extents)

Lat bounds: (3.772924193304, 11.318772579912)
Lon bounds: (-3.772924193304, 1.886462096652)
Time bounds: ['2013-04-13', '2018-01-01']


/home/localuser/Datacube/data_cube_notebooks/utils/data_cube_utilities/data_access_api.py:184: FutureWarning: casting an xarray.Dataset to a boolean will change in xarray v0.11 to only include data variables, not coordinates. Cast the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  if not dataset:


> #### Display Lat-Lon and Time Bounds

# Visualize Data Cube Region

> #### Picking a smaller analysis region

In [4]:
## The code below renders a map that can be used to orient yourself with the region.
from utils.data_cube_utilities.dc_display_map import display_map
display_map(latitude = latitude_extents, longitude = longitude_extents)

In [5]:
######### Vietnam - Buan Tua Srah Lake ################## 
longitude_extents = (108.02, 108.15)
latitude_extents  = (12.18 , 12.30)

# Ghana
longitude_extents = (0.1, 0.2)
latitude_extents = (7.0, 7.1)

time_extents = ('2015-01-01', '2016-01-01')
print ( time_extents )

('2015-01-01', '2016-01-01')


In [6]:
from utils.data_cube_utilities.dc_display_map import display_map

display_map(latitude = latitude_extents, longitude = longitude_extents)

In [7]:
landsat_dataset = dc.load(latitude = latitude_extents,
                          longitude = longitude_extents,
                          platform = platform,
                          time = time_extents,
                          product = product,
                          measurements = ['red', 'green', 'blue', 'nir', 'swir1', 'swir2', 'pixel_qa']) 

In [8]:
landsat_dataset

<xarray.Dataset>
Dimensions:    (latitude: 372, longitude: 372, time: 15)
Coordinates:
  * time       (time) datetime64[ns] 2015-01-04T10:15:20 ... 2015-12-22T10:15:26
  * latitude   (latitude) float64 7.1 7.1 7.099 7.099 ... 7.001 7.001 7.0 7.0
  * longitude  (longitude) float64 0.1001 0.1004 0.1007 ... 0.1996 0.1998 0.2001
Data variables:
    red        (time, latitude, longitude) int16 648 643 644 643 ... 570 569 570
    green      (time, latitude, longitude) int16 739 740 736 739 ... 714 713 707
    blue       (time, latitude, longitude) int16 562 564 561 562 ... 527 527 527
    nir        (time, latitude, longitude) int16 565 563 566 561 ... 461 461 461
    swir1      (time, latitude, longitude) int16 398 398 398 393 ... 342 343 343
    swir2      (time, latitude, longitude) int16 284 284 283 284 ... 256 259 260
    pixel_qa   (time, latitude, longitude) int32 324 324 324 324 ... 324 324 324
Attributes:
    crs:      EPSG:4326

# Derive Several Products

>### Unpack pixel_qa

In [9]:
import xarray as xr  
import numpy as np

def ls8_unpack_qa( data_array , cover_type):  
    
    land_cover_endcoding = dict( fill         =[1] ,
                                 clear        =[322, 386, 834, 898, 1346],
                                 water        =[324, 388, 836, 900, 1348],
                                 shadow       =[328, 392, 840, 904, 1350],
                                 snow         =[336, 368, 400, 432, 848, 880, 812, 944, 1352],
                                 cloud        =[352, 368, 416, 432, 848, 880, 912, 944, 1352],
                                 low_conf_cl  =[322, 324, 328, 336, 352, 368, 834, 836, 840, 848, 864, 880],
                                 med_conf_cl  =[386, 388, 392, 400, 416, 432, 898, 900, 904, 928, 944],
                                 high_conf_cl =[480, 992],
                                 low_conf_cir =[322, 324, 328, 336, 352, 368, 386, 388, 392, 400, 416, 432, 480],
                                 high_conf_cir=[834, 836, 840, 848, 864, 880, 898, 900, 904, 912, 928, 944], 
                                 terrain_occ  =[1346,1348, 1350, 1352]
                               )
    
    boolean_mask = np.isin(data_array, land_cover_endcoding[cover_type]) 
    return xr.DataArray(boolean_mask.astype(int),
                        coords = data_array.coords,
                        dims = data_array.dims,
                        name = cover_type + "_mask",
                        attrs = data_array.attrs)   

In [10]:
clear_xarray  = ls8_unpack_qa(landsat_dataset.pixel_qa, "clear")  
water_xarray  = ls8_unpack_qa(landsat_dataset.pixel_qa, "water")

shadow_xarray = ls8_unpack_qa(landsat_dataset.pixel_qa, "shadow")  

In [11]:
clean_xarray = xr.ufuncs.logical_or(clear_xarray , water_xarray).astype(np.int8).rename("clean_mask")

clean_mask = np.logical_or(clear_xarray.values.astype(bool),
                           water_xarray.values.astype(bool)) 

> ### Water

In [12]:
from utils.data_cube_utilities.dc_water_classifier import wofs_classify

water_classification = wofs_classify(landsat_dataset,
                                     clean_mask = clean_mask, 
                                     mosaic = False) 

In [13]:
wofs_xarray = water_classification.wofs

> ###  Normalized Indices  

In [14]:
def NDVI(dataset):
    return ((dataset.nir - dataset.red)/(dataset.nir + dataset.red)).rename("NDVI")

In [15]:
def NDWI(dataset):
    return ((dataset.green - dataset.nir)/(dataset.green + dataset.nir)).rename("NDWI")

In [16]:
def NDBI(dataset):
        return ((dataset.swir2 - dataset.nir)/(dataset.swir2 + dataset.nir)).rename("NDBI")

In [17]:
ndbi_xarray = NDBI(landsat_dataset)  # Urbanization - Reds
ndvi_xarray = NDVI(landsat_dataset)  # Dense Vegetation - Greens
ndwi_xarray = NDWI(landsat_dataset)  # High Concentrations of Water - Blues  

>### TSM  

In [18]:
from utils.data_cube_utilities.dc_water_quality import tsm

tsm_xarray = tsm(landsat_dataset).tsm#, clean_mask = wofs_xarray.values.astype(bool) ).tsm

> ### EVI  

In [19]:
def EVI(dataset, c1 = None, c2 = None, L = None):
        return ((dataset.nir - dataset.red)/((dataset.nir  + (c1 * dataset.red) - (c2 *dataset.blue) + L))).rename("EVI")

In [20]:
evi_xarray = EVI(landsat_dataset, c1 = 6, c2 = 7.5, L = 1 )

# Combine Everything  

In [21]:
combined_dataset = xr.merge([landsat_dataset,
          clean_xarray,
          clear_xarray,
          water_xarray,
          shadow_xarray,
          evi_xarray,
          ndbi_xarray,
          ndvi_xarray,
          ndwi_xarray,
          wofs_xarray,
          tsm_xarray])

# Copy original crs to merged dataset 
combined_dataset = combined_dataset.assign_attrs(landsat_dataset.attrs)

combined_dataset

<xarray.Dataset>
Dimensions:      (latitude: 372, longitude: 372, time: 15)
Coordinates:
  * time         (time) datetime64[ns] 2015-01-04T10:15:20 ... 2015-12-22T10:15:26
  * latitude     (latitude) float64 7.1 7.1 7.099 7.099 ... 7.001 7.001 7.0 7.0
  * longitude    (longitude) float64 0.1001 0.1004 0.1007 ... 0.1998 0.2001
Data variables:
    red          (time, latitude, longitude) float32 648.0 643.0 ... 569.0 570.0
    green        (time, latitude, longitude) float32 739.0 740.0 ... 713.0 707.0
    blue         (time, latitude, longitude) float32 562.0 564.0 ... 527.0 527.0
    nir          (time, latitude, longitude) float32 565.0 563.0 ... 461.0 461.0
    swir1        (time, latitude, longitude) float32 398.0 398.0 ... 343.0 343.0
    swir2        (time, latitude, longitude) float32 284.0 284.0 ... 259.0 260.0
    pixel_qa     (time, latitude, longitude) int32 324 324 324 ... 324 324 324
    clean_mask   (time, latitude, longitude) int8 1 1 1 1 1 1 1 ... 1 1 1 1 1 1
    clear_m

# Export Geotiff

----  
File formatting  

In [22]:
import time
def time_to_string(t):
    return time.strftime("%Y_%m_%d_%H_%M_%S", time.gmtime(t.astype(int)/1000000000))

----  
Start Export

In [23]:
# Ensure the output directory exists before writing to it.
!mkdir -p output/geotiffs/landsat8

from utils.data_cube_utilities.import_export import export_xarray_to_geotiff
export_xarray_to_geotiff(combined_dataset, "output/geotiffs/landsat8/landsat8")

----  
Sanity check using `gdalinfo` to make sure that all of our bands exist    

In [24]:
!gdalinfo output/geotiffs/landsat8/landsat8_2015_01_01_03_07_41.tif

ERROR 4: output/geotiffs/landsat8/landsat8_2015_01_01_03_07_41.tif: No such file or directory
gdalinfo failed - unable to open 'output/geotiffs/landsat8/landsat8_2015_01_01_03_07_41.tif'.


----  
Check to see what files exist in `geotiffs`

In [25]:
!ls -lah output/geotiffs/landsat8/*.tif

-rw-rw-r-- 1 localuser localuser 9.0M Nov  2 19:15 output/geotiffs/landsat8/landsat8_2015_01_04_10_15_20.tif
-rw-rw-r-- 1 localuser localuser 9.0M Nov  2 19:15 output/geotiffs/landsat8/landsat8_2015_01_20_10_15_16.tif
-rw-rw-r-- 1 localuser localuser 9.0M Nov  2 19:15 output/geotiffs/landsat8/landsat8_2015_02_05_10_15_13.tif
-rw-rw-r-- 1 localuser localuser 9.0M Nov  2 19:15 output/geotiffs/landsat8/landsat8_2015_02_21_10_15_06.tif
-rw-rw-r-- 1 localuser localuser 9.0M Nov  2 19:15 output/geotiffs/landsat8/landsat8_2015_03_09_10_14_57.tif
-rw-rw-r-- 1 localuser localuser 9.0M Nov  2 19:15 output/geotiffs/landsat8/landsat8_2015_04_10_10_14_40.tif
-rw-rw-r-- 1 localuser localuser 9.0M Nov  2 19:15 output/geotiffs/landsat8/landsat8_2015_04_26_10_14_36.tif
-rw-rw-r-- 1 localuser localuser 9.0M Nov  2 19:15 output/geotiffs/landsat8/landsat8_2015_05_12_10_14_20.tif
-rw-rw-r-- 1 localuser localuser 9.0M Nov  2 19:15 output/geotiffs/landsat8/landsat8_2015_05_28_10_14_22.tif
-rw-rw-r-- 1 localu

----  
Zip all geotiffs  

In [26]:
!tar -cvzf output/geotiffs/landsat_8/landsat8.tar.gz output/geotiffs/landsat8/*.tif

output/geotiffs/landsat8/landsat8_2015_01_04_10_15_20.tif
tar (child): output/geotiffs/landsat_8/landsat8.tar.gz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: output/geotiffs/landsat_8/landsat8.tar.gz: Wrote only 4096 of 10240 bytes
tar: Child returned status 2
tar: Error is not recoverable: exiting now


----  
List files again to see the size of the gif created

In [27]:
!ls -lah output/geotiffs/landsat8/*.tif

-rw-rw-r-- 1 localuser localuser 9.0M Nov  2 19:15 output/geotiffs/landsat8/landsat8_2015_01_04_10_15_20.tif
-rw-rw-r-- 1 localuser localuser 9.0M Nov  2 19:15 output/geotiffs/landsat8/landsat8_2015_01_20_10_15_16.tif
-rw-rw-r-- 1 localuser localuser 9.0M Nov  2 19:15 output/geotiffs/landsat8/landsat8_2015_02_05_10_15_13.tif
-rw-rw-r-- 1 localuser localuser 9.0M Nov  2 19:15 output/geotiffs/landsat8/landsat8_2015_02_21_10_15_06.tif
-rw-rw-r-- 1 localuser localuser 9.0M Nov  2 19:15 output/geotiffs/landsat8/landsat8_2015_03_09_10_14_57.tif
-rw-rw-r-- 1 localuser localuser 9.0M Nov  2 19:15 output/geotiffs/landsat8/landsat8_2015_04_10_10_14_40.tif
-rw-rw-r-- 1 localuser localuser 9.0M Nov  2 19:15 output/geotiffs/landsat8/landsat8_2015_04_26_10_14_36.tif
-rw-rw-r-- 1 localuser localuser 9.0M Nov  2 19:15 output/geotiffs/landsat8/landsat8_2015_05_12_10_14_20.tif
-rw-rw-r-- 1 localuser localuser 9.0M Nov  2 19:15 output/geotiffs/landsat8/landsat8_2015_05_28_10_14_22.tif
-rw-rw-r-- 1 localu